In [36]:
import os
import csv
from supabase import create_client, Client
import os
import datetime
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [61]:
COMPETITION_ID = 5
POOL_ID = 14
TODAY = datetime.date.today()
CURRENT_ROUND = 3


In [2]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            print(row)
            if row != {'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}:
                game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [57]:
round = "3"
update_game_schedule(f"2024/game-schedule-round-{round}-2024-ncaa-tournament.csv")

{'game_date': '2024-03-23', 'team_2_id': 'dayton', 'team_1_id': 'arizona', 'game_time': '12:45', 'round_num': '3', 'competition_id': '5'}
{'game_date': '2024-03-23', 'team_2_id': 'gonzaga', 'team_1_id': 'kansas', 'game_time': '15:15', 'round_num': '3', 'competition_id': '5'}
{'game_date': '2024-03-23', 'team_2_id': 'michigan-state', 'team_1_id': 'north-carolina', 'game_time': '17:30', 'round_num': '3', 'competition_id': '5'}
{'game_date': '2024-03-23', 'team_2_id': 'washington-state', 'team_1_id': 'iowa-state', 'game_time': '18:10', 'round_num': '3', 'competition_id': '5'}
{'game_date': '2024-03-23', 'team_2_id': 'oakland', 'team_1_id': 'north-carolina-state', 'game_time': '19:10', 'round_num': '3', 'competition_id': '5'}
{'game_date': '2024-03-23', 'team_2_id': 'texas', 'team_1_id': 'tennessee', 'game_time': '20:00', 'round_num': '3', 'competition_id': '5'}
{'game_date': '2024-03-23', 'team_2_id': 'duquesne', 'team_1_id': 'illinois', 'game_time': '20:40', 'round_num': '3', 'competitio

In [54]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = COMPETITION_ID, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    print(data)
    df = pd.DataFrame(data.data)
    df['points'] = None
    df['lost'] = None
    df['inactive'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"2024/{date}-game-scoring-2024-ncaa-tournament.csv", index=False, columns=['game_time', 'team_unique', 'lost', 'player_unique', 'points','inactive','game_id'])
    # 'rebounds', 'assists'])

In [62]:
# date = datetime.date.today()
date = "2024-03-23"
date = TODAY
generate_game_scoring_sheet(competition_id=COMPETITION_ID, date=date)

data=[{'player_unique': 'kj-simpson-1', 'team_unique': 'colorado', 'game_id': 243, 'game_date': '2024-03-24', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 5}, {'player_unique': 'tristan-dasilva-1', 'team_unique': 'colorado', 'game_id': 243, 'game_date': '2024-03-24', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 5}, {'player_unique': 'jvonne-hadley-1', 'team_unique': 'colorado', 'game_id': 243, 'game_date': '2024-03-24', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 5}, {'player_unique': 'eddie-lampkinjr-1', 'team_unique': 'colorado', 'game_id': 243, 'game_date': '2024-03-24', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 5}, {'player_unique': 'cody-williams-2', 'team_unique': 'colorado', 'game_id': 243, 'game_date': '2024-03-24', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 5}, {'player_unique': 'luke-obrien-1', 'team_unique': 'colorado', 'game_id': 243, 'game_date': '2024-03-24', 'game_time': '12:10:00', 'round_num': 

In [56]:
print(TODAY)

2024-03-23


In [22]:
def update_scores_from_csv(current_round: int, competition_id: int, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    losing_teams = []
    inactive_players = []
    team_1 = ""
    team_2 = ""
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            if row['lost'].upper() == 'L':
                losing_teams.append(row['team_unique'])
            if row['inactive'].upper() == 'I':
                inactive_players.append(row['player_unique'])
            if row['points'] != '':
                team_2 = team_1
                team_1 = row['team_unique']
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    for team in losing_teams:
        loser_data = supabase.table('team_competition').update({'round_eliminated': current_round}).eq('team_unique', team).eq('competition_id', competition_id).execute()
        print("losers")
        print(loser_data)
    for player in inactive_players:
        player_data = supabase.table('player_competition').update({'inactive': True}).eq('player_unique', player).eq('competition_id', competition_id).execute()
        print("inactive players")
        print(player_data)
    print(data)

In [65]:
# date = datetime.date.today()
current_round = CURRENT_ROUND
date = TODAY
update_scores_from_csv(current_round=current_round, competition_id=COMPETITION_ID, game_scoring_csv = f"2024/{date}-game-scoring-2024-ncaa-tournament.csv")


losers
data=[{'team_unique': 'colorado', 'competition_id': 5, 'league_unique': 'ncaambb', 'seed': 10, 'overall_seed': 39, 'region': 'SOUTH', 'round_eliminated': 3, 'round_started': 1, 'team_stats': {'pace': 69.1, 'conference': 'pac-12', 'offensive_rating': 113.6, 'assist_percentage': 56.1, 'turnover_percentage': 15.9, 'strength_of_schedule': 7.9, 'free_throw_attempt_rate': 0.353, 'three_point_attempt_rate': 0.304, 'true_shooting_percentage': 0.591, 'effective_field_goal_percentage': 0.553, 'two_point_field_goal_percentage': 0.536, 'three_point_field_goal_percentage': 0.394, 'opp_effective_field_goal_percentage': 0.493}, 'stats_thru': '2024-03-17', 'team_win_loss': {'wins': 24.0, 'losses': 10.0, 'games_played': 0, 'conference_wins': 13.0, 'conference_losses': 7.0}}] count=None
losers
data=[{'team_unique': 'utah-state', 'competition_id': 5, 'league_unique': 'ncaambb', 'seed': 8, 'overall_seed': 30, 'region': 'MIDWEST', 'round_eliminated': 3, 'round_started': 2, 'team_stats': {'pace': 69.

In [64]:
print(CURRENT_ROUND)

3
